In [43]:
from math import floor

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [44]:
df = pd.read_csv("data/2019/Divvy_Trips_2019_full.csv.gz", compression="gzip")

C:\Users\79500\AppData\Local\Temp\ipykernel_19416\672813569.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/2019/Divvy_Trips_2019_full.csv.gz", compression="gzip")


In [45]:
df.head()

,trip_id,start_time,end_time,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear
0,21742443,2019-01-01 00:04:37,2019-01-01 00:11:07,2167,390.0,199,Wabash Ave & Grand Ave,84,Milwaukee Ave & Grand Ave,Subscriber,Male,1989.0
1,21742444,2019-01-01 00:08:13,2019-01-01 00:15:34,4386,441.0,44,State St & Randolph St,624,Dearborn St & Van Buren St (*),Subscriber,Female,1990.0
2,21742445,2019-01-01 00:13:23,2019-01-01 00:27:12,1524,829.0,15,Racine Ave & 18th St,644,Western Ave & Fillmore St (*),Subscriber,Female,1994.0
3,21742446,2019-01-01 00:13:45,2019-01-01 00:43:28,252,"1,783.0",123,California Ave & Milwaukee Ave,176,Clark St & Elm St,Subscriber,Male,1993.0
4,21742447,2019-01-01 00:14:52,2019-01-01 00:20:56,1170,364.0,173,Mies van der Rohe Way & Chicago Ave,35,Streeter Dr & Grand Ave,Subscriber,Male,1994.0


# Потерянные велосипеды
Сгруппируем каждый датасет по bikeid и возьмем первую и последнюю поездку. После этого объеденим все датасеты в один и найдем самую последнюю поездку для каждого велосипеда. Вычтем из общего количества велосипедов велосипеды, последняя активность которых была в 2018-2019 => Узнаем кол-во потерянных/сломанных устройств

In [46]:
grouped_by_bike_2019 = (
    df.groupby("bikeid").agg({"start_time": "min", "end_time": "max"}).reset_index()
)

In [47]:
df_2 = pd.read_csv("data/2018/Divvy_Trips_2018_full.csv.gz", compression="gzip")
grouped_by_bike_2018 = df_2.groupby("01 - Rental Details Bike ID").agg(
    {
        "01 - Rental Details Local Start Time": "min",
        "01 - Rental Details Local End Time": "max",
    }
)
grouped_by_bike_2018 = grouped_by_bike_2018.reset_index()
grouped_by_bike_2018["01 - Rental Details Bike ID"] = grouped_by_bike_2018[
    "01 - Rental Details Bike ID"
].astype(int)
grouped_by_bike_2018 = grouped_by_bike_2018.rename(
    columns={
        "01 - Rental Details Bike ID": "bikeid",
        "01 - Rental Details Local Start Time": "start_time",
        "01 - Rental Details Local End Time": "end_time",
    }
)

C:\Users\79500\AppData\Local\Temp\ipykernel_19416\4142244348.py:1: DtypeWarning: Columns (1,2,4,6,8,9,10,13,14,18,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2 = pd.read_csv("data/2018/Divvy_Trips_2018_full.csv.gz", compression="gzip")


In [48]:
df_2 = pd.read_csv("data/2017/Divvy_Trips_2017_full.csv.gz", compression="gzip")
grouped_by_bike_2017 = (
    df_2.groupby("bikeid").agg({"start_time": "min", "end_time": "max"}).reset_index()
)

In [49]:
df_2 = pd.read_csv("data/2016/Divvy_Trips_2016_full.csv.gz", compression="gzip")
grouped_by_bike_2016 = (
    df_2.groupby("bikeid")
    .agg({"starttime": "min", "stoptime": "max"})
    .rename(columns={"starttime": "start_time", "stoptime": "end_time"})
    .reset_index()
)

In [50]:
df_2 = pd.read_csv("data/2015/Divvy_Trips_2015_full.csv.gz", compression="gzip")
grouped_by_bike_2015 = (
    df_2.groupby("bikeid")
    .agg({"starttime": "min", "stoptime": "max"})
    .rename(columns={"starttime": "start_time", "stoptime": "end_time"})
    .reset_index()
)

In [51]:
df_2 = pd.read_csv("data/2014/Divvy_Trips_2014_full.csv.gz", compression="gzip")
grouped_by_bike_2014 = (
    df_2.groupby("bikeid")
    .agg({"starttime": "min", "stoptime": "max"})
    .rename(columns={"starttime": "start_time", "stoptime": "end_time"})
    .reset_index()
)

In [52]:
full_grouped = pd.concat(
    [
        grouped_by_bike_2014,
        grouped_by_bike_2015,
        grouped_by_bike_2016,
        grouped_by_bike_2017,
        grouped_by_bike_2018,
        grouped_by_bike_2019,
    ]
)

In [53]:
full_grouped["start_time"] = pd.to_datetime(full_grouped["start_time"])
full_grouped["end_time"] = pd.to_datetime(full_grouped["end_time"])

In [54]:
full_grouped.head()

,bikeid,start_time,end_time
0,1,2014-01-10 15:02:00,2014-09-09 08:44:00
1,2,2014-10-01 16:28:00,2014-09-09 08:26:00
2,3,2014-01-30 16:59:00,2014-09-30 12:55:00
3,4,2014-01-14 08:00:00,2014-07-09 18:22:00
4,5,2014-10-28 18:01:00,2014-09-09 18:19:00


In [55]:
# Получаем датасет с самой первой и последней поездкой для каждого велосипеда
full_grouped_result = full_grouped.groupby("bikeid").agg(
    {"start_time": "min", "end_time": "max"}
)

In [56]:
# Убираем выбросы - не может быть так, что первая поездка была позже последней
full_grouped_result = full_grouped_result[
    full_grouped_result["start_time"] < full_grouped_result["end_time"]
]

In [57]:
full_grouped_result["end_time"] = full_grouped_result["end_time"].dt.year
full_grouped_result_2019 = full_grouped_result[full_grouped_result["end_time"] >= 2018]

full_grouped_result_2019

,start_time,end_time
bikeid,,
1,2014-01-10 15:02:00,2018
2,2014-10-01 16:28:00,2018
3,2014-01-30 16:59:00,2018
4,2014-01-14 08:00:00,2018
5,2014-10-28 18:01:00,2018
...,...,...
6467,2017-10-01 11:56:00,2018
6468,2017-10-01 14:08:00,2018
6469,2017-10-01 11:42:00,2018


In [60]:
lost_bicycle = full_grouped_result.shape[0] - full_grouped_result_2019.shape[0]
lost_bicycle # Кол-во потерянных или сломанных велосипедов в период с 2013 по 2018-2019

703

6354